### Установка и импорт необходимых пакетов

In [1]:
! pip install pymystem3 &>/dev/null

In [2]:
import warnings
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import auc, roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

nltk.download("stopwords")
stop_words = stopwords.words("russian")

warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


---

### Обработка текста

In [3]:
df = pd.read_csv('financial_review.csv')

In [4]:
df.shape

(23810, 10)

In [5]:
lemmatizer = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [6]:
all_tokens = []
def clean_text(text):
    text = re.sub(r"[^\w\s]", "", text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token)[0] for token in text.split(" ") if token.isalpha()]
    text = [word for word in text if not word in stop_words]
    all_tokens.extend(text)
    text = " ".join(text)
    return text

In [7]:
df["processed_review"] = df["review"].apply(lambda x: clean_text(x))

In [8]:
df[['processed_review']].iloc[1][0]

'годумный оформлять мега карта тц мегадыбенко сотрудник зао кредит евпроп банк оформлять бонусный карта начислять приветственный бонус кредитный лимит составлять ноль рубль ноль копейка бонус потратить карта выбрасывать забывать существование срок действие карта истекать год год уходить декретный отпуск далее отпуск уход ребенок находиться немой сей день май год узнавать долг карта составлять рубль семь тысячтрист шестьдесят восемь рубль копейка повзвонить банк узнавать начислять штраф оплата комиссия предоставление почтовый выписка размер рубль никакой почтовый выписка карта рука получать задолженность знать считать данный задолженность необоснованный штраф превышать сумма долг данный задолженность никакой выписка получать звонок сотрудник банка домашний телефон оставаться просить закрывать договор данный связь условие продукт тариф банка вопрос решать близкий второй экземпляр жалоба направлять роспотребнадзор осуществление копия жалоба направлять приемная банка оформлять мега карта т

In [9]:
mapping = {'1': 0, '2': 0, '3': None, '4': 1, '5': 1} # score = 3 считаем нейтральным
df['score'] = df['score'].map(mapping)

In [10]:
df = df[~df['score'].isna()] # удаляем строки с неизвестным или нейтральным score

In [11]:
df['score'].value_counts()

0.0    11484
1.0     4291
Name: score, dtype: int64

In [12]:
y = df["score"].tolist()
X = df["processed_review"].tolist()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) # разбиваем данные на тренировочную и тестовую выборки

---

### BoW + Логистическая регрессия

In [14]:
# пайплайн для определения оптимальных параметров CountVectorizer с LogReg
pipeline_bow_logreg = Pipeline([
           ('vect', CountVectorizer()),
           ('clf', LogisticRegression(random_state=0)),
])

In [15]:
# задание параметров, по которым будет производиться поиск
parameters = [{
    'vect__max_df': (1, 0.95),
    'vect__min_df': (0, 0.05),
    'vect__ngram_range': ((1, 1), (1, 2), (2, 2)),
}]

In [16]:
grid_search_bow_logreg = GridSearchCV(pipeline_bow_logreg, parameters)

In [17]:
grid_search_bow_logreg.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             param_grid=[{'vect__max_df': (1, 0.95), 'vect__min_df': (0, 0.05),
                          'vect__ngram_range': ((1, 1), (1, 2), (2, 2))}])

In [18]:
grid_search_bow_logreg.best_estimator_.get_params # набор оптимальных параметров

<bound method Pipeline.get_params of Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.95, min_df=0, ngram_range=(1, 2))),
                ('clf', LogisticRegression(random_state=0))])>

In [19]:
# отсекаются наиболее частые термы, используются униграммы и биграммы
vectorizer_bow_logreg = CountVectorizer(max_df=0.95, min_df=0, ngram_range=(1, 2))

In [20]:
vectorizer_bow_logreg.fit(X_train)

CountVectorizer(max_df=0.95, min_df=0, ngram_range=(1, 2))

In [21]:
X_train_bow = vectorizer_bow_logreg.transform(X_train)
X_test_bow = vectorizer_bow_logreg.transform(X_test)

In [22]:
logreg_bow = LogisticRegression(random_state=0) # логистическая регрессия с BoW
logreg_bow.fit(X_train_bow, y_train)
y_pred_bow = logreg_bow.predict(X_test_bow)
y_pred_prob_bow = logreg_bow.predict_proba(X_test_bow)[:, 1]

---

### TF-IDF + логистическая регрессия

In [23]:
# пайплайн для определения оптимальных параметров TfidfVectorizer с LogReg
pipeline_tfidf_logreg = Pipeline([
           ('vect', TfidfVectorizer()),
           ('clf', LogisticRegression(random_state=0)),
])

In [24]:
grid_search_tfidf_logreg = GridSearchCV(pipeline_tfidf_logreg, parameters)

In [25]:
grid_search_tfidf_logreg.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             param_grid=[{'vect__max_df': (1, 0.95), 'vect__min_df': (0, 0.05),
                          'vect__ngram_range': ((1, 1), (1, 2), (2, 2))}])

In [26]:
grid_search_tfidf_logreg.best_estimator_.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('vect', TfidfVectorizer(max_df=0.95, min_df=0)),
                ('clf', LogisticRegression(random_state=0))])>

In [27]:
# отсекаются наиболее частые термы, используются только униграммы
vectorizer_tfidf_logreg = TfidfVectorizer(max_df=0.95, min_df=0, ngram_range=(1, 1))

In [28]:
vectorizer_tfidf_logreg.fit(X_train)

TfidfVectorizer(max_df=0.95, min_df=0)

In [29]:
X_train_tfidf = vectorizer_tfidf_logreg.transform(X_train)
X_test_tfidf = vectorizer_tfidf_logreg.transform(X_test)

In [30]:
logreg_tfidf = LogisticRegression(random_state=0) # логистическая регрессия с TF-IDF
logreg_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = logreg_tfidf.predict(X_test_tfidf)
y_pred_prob_tfidf = logreg_tfidf.predict_proba(X_test_tfidf)[:, 1]

---

### ROC-кривая и матрица ошибок для LogReg 


In [31]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_shape(type="line", line=dict(dash="dash"), x0=0, x1=1, y0=0, y1=1)
name = "BoW"
fpr, tpr, _ = roc_curve(y_test, y_pred_prob_bow)
auc_value = round(auc(fpr, tpr), 2)
fig.add_trace(go.Scatter(x=fpr, y=tpr, name=f"{name}: {auc_value}", mode="lines"))

name = "TF-IDF"
fpr, tpr, _ = roc_curve(y_test, y_pred_prob_tfidf)
auc_value = round(auc(fpr, tpr), 2)
fig.add_trace(go.Scatter(x=fpr, y=tpr, name=f"{name}: {auc_value}", mode="lines"))

fig.update_layout(
    width=500,
    height=500,
    template="plotly_white",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    title="Logistic Regression",
    legend=dict(yanchor="bottom", y=0.01, xanchor="right", x=0.99),
)

fig.show()

In [32]:
confusion_matrix(y_test, y_pred_bow)

array([[2821,   62],
       [ 166,  895]])

In [33]:
confusion_matrix(y_test, y_pred_tfidf)

array([[2854,   29],
       [ 203,  858]])

---

---

### BoW + XGBClassifier

In [34]:
# пайплайн для определения оптимальных параметров CountVectorizer с XGBClassifier
pipeline_bow_xgb = Pipeline([
           ('vect', CountVectorizer()),
           ('clf', XGBClassifier(random_state=0)),
])

In [35]:
grid_search_bow_xgb = GridSearchCV(pipeline_bow_xgb, parameters)

In [36]:
grid_search_bow_xgb.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('clf', XGBClassifier())]),
             param_grid=[{'vect__max_df': (1, 0.95), 'vect__min_df': (0, 0.05),
                          'vect__ngram_range': ((1, 1), (1, 2), (2, 2))}])

In [37]:
grid_search_bow_xgb.best_estimator_.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.95, min_df=0, ngram_range=(1, 2))),
                ('clf', XGBClassifier())])>

In [38]:
# отсекаются наиболее частые термы, используются униграммы и биграммы
vectorizer_bow_xgb = CountVectorizer(max_df=0.95, min_df=0, ngram_range=(1, 2))

In [39]:
vectorizer_bow_xgb.fit(X_train)

CountVectorizer(max_df=0.95, min_df=0, ngram_range=(1, 2))

In [40]:
X_train_bow = vectorizer_bow_xgb.transform(X_train)
X_test_bow = vectorizer_bow_xgb.transform(X_test)

In [41]:
xgb_bow = XGBClassifier(random_state=0) # XGBClassifier с BoW
xgb_bow.fit(X_train_bow, y_train)
y_pred_bow = xgb_bow.predict(X_test_bow)
y_pred_prob_bow = xgb_bow.predict_proba(X_test_bow)[:, 1]

---

### TF-IDF + XGBClassifier


In [42]:
# пайплайн для определения оптимальных параметров TfidfVectorizer с XGBClassifier
pipeline_tfidf_xgb = Pipeline([
           ('vect', TfidfVectorizer()),
           ('clf', XGBClassifier(random_state=0)),
])

In [43]:
grid_search_tfidf_xgb = GridSearchCV(pipeline_tfidf_xgb, parameters)

In [44]:
grid_search_tfidf_xgb.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf', XGBClassifier())]),
             param_grid=[{'vect__max_df': (1, 0.95), 'vect__min_df': (0, 0.05),
                          'vect__ngram_range': ((1, 1), (1, 2), (2, 2))}])

In [45]:
grid_search_tfidf_xgb.best_estimator_.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('vect',
                 TfidfVectorizer(max_df=0.95, min_df=0, ngram_range=(1, 2))),
                ('clf', XGBClassifier())])>

In [46]:
# отсекаются наиболее частые термы, используются униграммы и биграммы
vectorizer_tfidf_xgb = TfidfVectorizer(max_df=0.95, min_df=0, ngram_range=(1, 2))

In [47]:
vectorizer_tfidf_xgb.fit(X_train)

TfidfVectorizer(max_df=0.95, min_df=0, ngram_range=(1, 2))

In [48]:
X_train_tfidf = vectorizer_tfidf_xgb.transform(X_train)
X_test_tfidf = vectorizer_tfidf_xgb.transform(X_test)

In [49]:
xgb_tfidf = XGBClassifier(random_state=0) # XGBClassifier с TF-IDF
xgb_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = xgb_tfidf.predict(X_test_tfidf)
y_pred_prob_tfidf = xgb_tfidf.predict_proba(X_test_tfidf)[:, 1]

---

### ROC-кривая и матрица ошибок для XGBClassifier

In [53]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_shape(type="line", line=dict(dash="dash"), x0=0, x1=1, y0=0, y1=1)
name = "BoW"
fpr, tpr, _ = roc_curve(y_test, y_pred_prob_bow)
auc_value = round(auc(fpr, tpr), 2)
fig.add_trace(go.Scatter(x=fpr, y=tpr, name=f"{name}: {auc_value}", mode="lines"))

name = "TF-IDF"
fpr, tpr, _ = roc_curve(y_test, y_pred_prob_tfidf)
auc_value = round(auc(fpr, tpr), 2)
fig.add_trace(go.Scatter(x=fpr, y=tpr, name=f"{name}: {auc_value}", mode="lines"))

fig.update_layout(
    width=500,
    height=500,
    template="plotly_white",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    title="XGBClassifier",
    legend=dict(yanchor="bottom", y=0.01, xanchor="right", x=0.99),
)

fig.show()

In [51]:
confusion_matrix(y_test, y_pred_bow)

array([[2831,   52],
       [ 321,  740]])

In [52]:
confusion_matrix(y_test, y_pred_tfidf)

array([[2804,   79],
       [ 273,  788]])

Логистическая регрессия показала себя немного лучше, чем XGBClassifier. Использование униграмм и биграмм вместе, а также отсечение самых частых термов, дало прирост производительности. При этом оба алгоритма классификации показали схожие результаты при использовании различных методов векторизации текста. Стоит отметить, что при векторизации с параметрами по умолчанию лучше себя показал TF-IDF на обоих алгоритмах классификации.